In [1]:
import pandas as pd

# 加载上传的文件
file_path = '/Volumes/QC/INT/INT_BN246_HC135BP_allMDD/Results/INTvalue_HCMDD.csv'
data = pd.read_csv(file_path)

# 显示数据的前几行，检查文件内容和结构
data.head(), data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Columns: 248 entries, subID to lPFtha_R
dtypes: float64(246), int64(1), object(1)
memory usage: 1.3+ MB


(            subID  disorder     A8m_L     A8m_R    A8dl_L    A8dl_R     A9l_L  \
 0  sub-1000008V01         0  3.819513  4.596507  4.288563  3.995476  4.192610   
 1  sub-1000018V01         0  3.755922  3.133401  3.050524  3.223097  3.278991   
 2  sub-1000022V01         0  3.709563  3.585366  3.588804  3.911602  3.454135   
 3  sub-1000027V01         0  3.078767  2.851947  3.281958  3.343382  3.532308   
 4  sub-1000032V01         0  3.352897  3.101042  3.643403  3.599043  3.864443   
 
       A9l_R    A6dl_L    A6dl_R  ...   rTtha_L   rTtha_R   PPtha_L   PPtha_R  \
 0  4.300439  3.868013  3.904747  ...  3.204395  2.851630  2.825454  2.931901   
 1  3.152930  3.102638  3.082079  ...  3.241501  2.571669  2.961889  2.709233   
 2  3.905066  3.813304  3.799678  ...  3.218293  3.316073  3.586145  3.613834   
 3  3.608725  3.583193  3.452513  ...  3.153778  3.398448  3.724971  3.619586   
 4  3.445477  3.706129  3.451918  ...  3.024197  3.554753  3.494057  3.314260   
 
      Otha_L    Ot

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 删除非数值特征
data_cleaned = data.drop(columns=['subID'])

# 分离特征和目标变量
X = data_cleaned.drop(columns=['disorder'])
y = data_cleaned['disorder']

In [3]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 使用随机森林模型进行初步分类
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [5]:
# 预测并计算准确率
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

accuracy, classification_report_output

(0.6642857142857143,
 '              precision    recall  f1-score   support\n\n           0       0.38      0.07      0.11        45\n           1       0.68      0.95      0.79        95\n\n    accuracy                           0.66       140\n   macro avg       0.53      0.51      0.45       140\nweighted avg       0.58      0.66      0.57       140\n')

In [6]:
from sklearn.utils import resample

# 平衡数据集：过采样类别较少的数据
data_balanced = pd.concat([
    resample(data_cleaned[data_cleaned['disorder'] == 0],
             replace=True,
             n_samples=data_cleaned[data_cleaned['disorder'] == 1].shape[0],
             random_state=42),
    data_cleaned[data_cleaned['disorder'] == 1]
])

In [8]:
# 分离特征和目标变量
X_balanced = data_balanced.drop(columns=['disorder'])
y_balanced = data_balanced['disorder']

# 重新划分训练集和测试集
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced)

# 使用优化参数的随机森林分类器
rf_model_bal = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=15, class_weight='balanced')
rf_model_bal.fit(X_train_bal, y_train_bal)


RandomForestClassifier(class_weight='balanced', max_depth=15, n_estimators=200,
                       random_state=42)

In [9]:
# 预测并计算新结果
y_pred_bal = rf_model_bal.predict(X_test_bal)
accuracy_bal = accuracy_score(y_test_bal, y_pred_bal)
classification_report_bal = classification_report(y_test_bal, y_pred_bal)

accuracy_bal, classification_report_bal

(0.868421052631579,
 '              precision    recall  f1-score   support\n\n           0       0.92      0.81      0.86        95\n           1       0.83      0.93      0.88        95\n\n    accuracy                           0.87       190\n   macro avg       0.87      0.87      0.87       190\nweighted avg       0.87      0.87      0.87       190\n')

In [ ]:
import pandas as pd
from sklearn.utils import resample